In [ ]:
import torch
print(torch.cuda.is_available() , torch.__version__)

In [1]:
from src2.api import Trainer  
app = Trainer.initialize(stage = 0 , resume = 0 , checkname= 1)

--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to booster_day_ShortTest!
StatusDisplay(verbosity=10) , display epoch and event information
DetailedAlphaAnalysis(use_num=avg) , record and concat each model to Alpha model instance


In [2]:
app.go()

24-09-07 10:20:21|MOD:display     |: Model Specifics:
24-09-07 10:20:21|MOD:display     |: Start Process [Data] at Sat Sep  7 10:20:21 2024!


{'model_name': 'booster_day_ShortTest',
 'model_module': 'booster',
 'model_types': None,
 'model_booster_type': 'lgbm',
 'model_booster_head': False,
 'model_data_types': ['day'],
 'model_data_labels': None,
 'random_seed': None,
 'beg_date': 20170101,
 'end_date': 20170228,
 'train_sample_method': 'sequential',
 'train_shuffle_option': 'epoch'}
{'seqlens': [{'day': 5}],
 'objective': ['regression'],
 'linear_tree': [True],
 'learning_rate': [0.3],
 'lambda_l2': [1e-05],
 'alpha': [1e-07],
 'num_leaves': [31],
 'max_depth': [6, 8, 10],
 'min_sum_hessian_in_leaf': [1],
 'feature_fraction': [0.6],
 'bagging_fraction': [0.75],
 'force_col_wise': [True],
 'monotone_constraints': [0],
 'verbosity': 10}
try using d:\Coding\learndl\learndl\data\DataSet/day.20240607.pt , success!


24-09-07 10:20:23|MOD:display     |: Finish Process [Data], Cost 2.4 Secs
24-09-07 10:20:23|MOD:display     |: Start Process [Fit] at Sat Sep  7 10:20:23 2024!
24-09-07 10:20:23|MOD:classes     |: First Iterance: (20170103 , 0)


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


100%|██████████| 10/10 [00:00<?, ?it/s]


...

,group,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
num,type,,,,,,,,,,,,,,,,,,,,
0,best,-2.480%,-2.035%,-0.872%,-1.275%,-1.061%,3.182%,-0.842%,0.586%,1.656%,1.939%,-0.941%,-0.968%,-0.937%,-0.896%,-0.844%,-0.657%,-0.305%,-0.162%,-0.282%,-1.064%
1,best,-2.038%,-1.998%,-1.227%,-1.151%,-0.974%,3.063%,-1.075%,-0.946%,1.657%,3.451%,-0.822%,-0.753%,-0.739%,-0.723%,-0.672%,-0.774%,-0.494%,-0.550%,-0.411%,-0.989%
2,best,-2.322%,-1.811%,-1.081%,-1.274%,0.757%,1.066%,-0.134%,1.920%,0.904%,-0.875%,-0.663%,0.748%,-0.864%,-0.603%,-0.718%,-0.743%,-0.625%,-0.272%,-0.388%,-1.118%


Grouped Return Results are saved to d:\Coding\learndl\learndl\models\booster_day_ShortTest\detailed_analysis\group.xlsx


,factor_name,benchmark,start,end,pf,bm,excess,annualized,mdd,te,ir,calmar,turnover,mdd_period
1,best,csi300,20170104,20170302,1.48%,1.97%,-0.49%,-3.33%,2.86%,5.34%,-0.624,-1.167,3.591,20170118-20170222
2,best,csi500,20170104,20170302,-0.36%,0.66%,-1.01%,-7.23%,3.79%,7.42%,-0.974,-1.908,3.565,20170118-20170302
0,best,csi1000,20170104,20170302,-0.21%,-2.29%,2.08%,12.43%,2.60%,9.23%,1.346,4.773,3.625,20170118-20170302


24-09-07 10:20:59|MOD:time        |: Main Process Finished! Cost 38.8 Seconds


Analytic datas are saved to d:\Coding\learndl\learndl\models\booster_day_ShortTest\detailed_analysis\data.xlsx
Analytic plots are saved to d:\Coding\learndl\learndl\models\booster_day_ShortTest\detailed_analysis\plot.pdf


In [54]:
torch.concat([dl[0].x , dl[1].x ]).shape

torch.Size([5043, 5, 6])

In [ ]:
from src.api import HiddenExtractor
extractor = HiddenExtractor('resnet_gru_30m' , model_nums=[1] , model_types=['best'])
extractor.extract_hidden('update' , deploy = True)

In [ ]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()

In [ ]:
from src.api import Trainer
Trainer.update_models()